In [1]:
import dependencies
import line_profiler
import numpy as np

from Scripts.generate_data import generate_Ys
from Scripts.scBiGLasso import analyticBiGLasso
from Scripts.utilities import generate_confusion_matrices
from Scripts.utilities import precision, recall, accuracy

np.set_printoptions(precision=3, suppress=True)
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

Python Version 3.9.12 (main, Apr  5 2022, 01:53:17) 
[Clang 12.0.0 ]
Numpy Version 1.21.5
Scipy Version 1.7.3
Sklearn Version 1.0.2


In [28]:
# Standard way to generate matrix variate data, takes ~1 second
Psi_gen, Theta_gen, Ys = generate_Ys(
    m=100,
    p=(p:=100),
    n=(n:=20),
    expected_nonzero_psi=n**2,#n**2 / 5, # (divide by 5 for sparsity)
    expected_nonzero_theta=p**2,#p**2 / 5,
    structure="Kronecker Sum"
)
Ys.shape

(100, 100, 20)

In [42]:
# Even when vindication is low, we still get excellent results?
Psi, Theta = analyticBiGLasso(
    Ys,
    beta_1 = 0.002,
    beta_2 = 0.02,
    vindicate = True
)
print("===Psi===")
print(Psi_cm := generate_confusion_matrices(Psi, Psi_gen, mode='Negative'))
print(
    f"{precision(Psi_cm)=:.3f}",
    f"\n{recall(Psi_cm)=:.3f}",
    f"\n{accuracy(Psi_cm)=:.3f}"
)
print("\n==Theta==")
print(Theta_cm := generate_confusion_matrices(Theta, Theta_gen, mode='Negative'))
print(
    f"{precision(Theta_cm)=:.3f}",
    f"\n{recall(Theta_cm)=:.3f}",
    f"\n{accuracy(Theta_cm)=:.3f}"
)

Psi vindication: 0.18028892294419974
Theta vindication: 0.9690547914837584
===Psi===
[[124.   0.]
 [ 82. 174.]]
precision(Psi_cm)=1.000 
recall(Psi_cm)=0.602 
accuracy(Psi_cm)=0.784

==Theta==
[[4096.  760.]
 [ 932. 4112.]]
precision(Theta_cm)=0.843 
recall(Theta_cm)=0.815 
accuracy(Theta_cm)=0.829


In [4]:
# You can get detailed runtime info on analyticBiGLasso by running this cell
kwargs = {
    "Ys": Ys,
    "beta_1": 0.02,
    "beta_2": 0.02,
}
%lprun -f analyticBiGLasso analyticBiGLasso(**kwargs)